In [1]:

import html2text

# FILEPATH: /Users/galen/Documents/Work/grand unification theory/gmail/export.ipynb
"""Shows basic usage of the Gmail API.
Lists the user's Gmail labels.
"""
import os.path
import base64
import email

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build


from googleapiclient.errors import HttpError


In [2]:

# If modifying these scopes, delete the file token.json.
SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']



In [6]:


creds = None
# The file token.json stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

try:
    # Call the Gmail API
    service = build('gmail', 'v1', credentials=creds)
    results = service.users().messages().list(userId='me', labelIds=['INBOX']).execute()
    msgs = results.get('messages', [])

    if not msgs:
        print('No messages found.')
    else:
        print('Messages found.')
        for msg in msgs[:10]:
            # get message content 
            message = service.users().messages().get(userId='me', id=msg['id']).execute()
            message_body = None
            h = html2text.HTML2Text()
            h.ignore_links = True
            h.ignore_images = True

            if 'payload' in message and 'parts' in message['payload']:
                for part in message['payload']['parts']:
                    if part['mimeType'] == 'text/plain':
                        message_body = base64.urlsafe_b64decode(part['body']['data'].encode('UTF-8')).decode('UTF-8')
                    elif part['mimeType'] == 'text/html':
                        message_body = h.handle(base64.urlsafe_b64decode(part['body']['data'].encode('UTF-8')).decode('UTF-8'))
            elif 'payload' in message and 'body' in message['payload']:
                if message['payload']['mimeType'] == 'text/plain':
                    message_body = base64.urlsafe_b64decode(message['payload']['body']['data'].encode('UTF-8')).decode('UTF-8')
                elif message['payload']['mimeType'] == 'text/html':
                    message_body = h.handle(base64.urlsafe_b64decode(message['payload']['body']['data'].encode('UTF-8')).decode('UTF-8'))
            if message_body:
                with open(f"messages/{msg['id']}.txt", "w") as f:
                    f.write(message_body)
            else:
                print(f"No message body found for message {msg['id']}")
                print(message)
                
except HttpError as error:
    print(f'An error occurred: {error}')


Messages found.
